In [1]:
import numpy as np
import pandas as pd
import time
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = "E:/CS_Master_Degree_UIUC/CS410_Text_Information_system/Project/Project Submission/CourseProject/Dataset/"
# Load transaction dataset
D = pd.read_csv(path + "DBLP2000.csv").astype('str')
D

,author,title
0,Lothar Breuer,Operator-Geometric Solutions for the M/G/k Que...
1,"Christopher Lusena, Judy Goldsmith, Martin Mun...",Nonapproximability Results for Partially Obser...
2,"János Komlós, Ali Shokoufandeh, Miklós Simonov...",The Regularity Lemma and Its Applications in G...
3,Vijay V. Vazirani,Primal-Dual Schema Based Approximation Algorit...
4,"Isabel Fernández-Anta, Eva Millán, José-Luis P...",Adaptation and Generation in a Web-Based Lisp ...
...,...,...
3999,"Marek Teichmann, Bud Mishra",Probabilistic Algorithms for Efficient Graspin...
4000,Robert E. Filman,Arachnoid Tourist: 2020 Hindsight.
4001,"Lassaâd Gannoun, Julien Francioli, Stanislav C...",Domain Name eXchange: A Mobile-Agent-Based Sha...
4002,"Heecheol Jeon, Charles J. Petrie, Mark R. Cutk...",JATLite: A Java Agent Infrastructure with Mess...


In [3]:
# Load context units
context_units = pd.read_csv(path + "DBLP2000_context_units.csv")
context_units

,pattern,transaction_index,pattern_type
0,Ralf Steinmetz,"[70, 1323, 1325, 3362]",author
1,Cheng-Wen Wu,"[194, 196, 288, 1909]",author
2,Thomas S. Huang,"[652, 835, 1012, 1123]",author
3,Maja J. Mataric,"[660, 669, 2266, 2271]",author
4,K. Suzanne Barber,"[707, 1311, 2129, 2444, 2448]",author
...,...,...,...
175,robocup,"[1381, 2200, 2202, 2819]",title
176,contextu,"[138, 156, 793, 993]",title
177,intension,"[272, 312, 990, 1360]",title
178,nest,"[636, 2289, 3280, 3402]",title


### Build weight matrix of transactions in the context unit space as bit vectors

In [4]:
D_size = len(D) # number of rows
C_size = len(context_units) # number of columns
# Initialize the weight matrix 
transaction_w = np.zeros([C_size, D_size])
context_index = context_units['transaction_index'].apply(lambda row: row[1:-1].split(', ')).apply(lambda row: list(map(int, row)))
for i in range(C_size):
    context_ind = context_index.iloc[i]
    transaction_w[i, context_ind] = 1

In [5]:
transaction_weights = pd.DataFrame(transaction_w, index = context_units["pattern"])
transaction_weights

,0,1,2,3,4,5,6,7,8,9,...,3994,3995,3996,3997,3998,3999,4000,4001,4002,4003
pattern,,,,,,,,,,,,,,,,,,,,,
Ralf Steinmetz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cheng-Wen Wu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Thomas S. Huang,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maja J. Mataric,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
K. Suzanne Barber,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
robocup,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
contextu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
intension,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
transaction_weights.to_csv(path + "transaction_weights.csv", index = False)

### Calculate similarity between the given transaction and the author 

In [6]:
# Get author list 
authors = context_units.iloc[0:14]["pattern"]
authors

0          Ralf Steinmetz
1            Cheng-Wen Wu
2         Thomas S. Huang
3         Maja J. Mataric
4       K. Suzanne Barber
5           Josef Kittler
6           Gerald Sommer
7        Edwin R. Hancock
8      Masaru Kitsuregawa
9        Roberto Gorrieri
10          Nancy E. Reed
11     Bharat K. Bhargava
12    Sanjay Kumar Madria
13           Bill Hancock
Name: pattern, dtype: object

In [7]:
# Get author weights
context_weights = pd.read_csv(path+ "Context_units_weights.csv")
author_weights = context_weights.iloc[:, 1:15]
author_weights

,Ralf Steinmetz,Cheng-Wen Wu,Thomas S. Huang,Maja J. Mataric,K. Suzanne Barber,Josef Kittler,Gerald Sommer,Edwin R. Hancock,Masaru Kitsuregawa,Roberto Gorrieri,Nancy E. Reed,Bharat K. Bhargava,Sanjay Kumar Madria,Bill Hancock
0,0.005007,0.003278,0.004386,0.003792,0.003458,0.003737,0.004386,0.004574,0.004262,0.004524,0.003418,0.003684,0.004867,0.002880
1,0.003278,0.005162,0.003851,0.003851,0.004071,0.002692,0.004453,0.004651,0.004330,0.004026,0.003477,0.004330,0.004330,0.003397
2,0.004386,0.003851,0.005007,0.003227,0.003458,0.003737,0.004386,0.004574,0.004262,0.004524,0.003418,0.004262,0.004867,0.003321
3,0.003792,0.003851,0.003227,0.004386,0.003458,0.002649,0.004386,0.004003,0.003684,0.003958,0.002903,0.003684,0.004262,0.002461
4,0.003458,0.004071,0.003458,0.003458,0.004684,0.003402,0.004003,0.004684,0.004434,0.004092,0.004092,0.003880,0.004434,0.002897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.003134,0.003743,0.003684,0.003134,0.003880,0.003086,0.003684,0.003880,0.003576,0.003835,0.003310,0.004138,0.004138,0.002759
176,0.003914,0.003382,0.003914,0.003331,0.003579,0.003858,0.004526,0.004734,0.003806,0.004684,0.004098,0.003806,0.005027,0.003018
177,0.004453,0.003910,0.003851,0.004453,0.004071,0.003230,0.004453,0.004651,0.004330,0.004601,0.003477,0.004330,0.004944,0.002947
178,0.003179,0.004390,0.003737,0.003179,0.003940,0.002609,0.003737,0.004502,0.004198,0.003895,0.003363,0.004198,0.004198,0.003250


In [8]:
# Define the cosine similarity between two vectors 
def cos_sim(a, b):
    """
    Takes 2 vectors a, b and returns the cosine similarity 
    """
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    return dot_product / (norm_a * norm_b)

In [9]:
def sim_tran_pattern(transaction_weights, pattern_weights):
    
    num_tran = transaction_weights.shape[1]
    num_p = pattern_weights.shape[1]
    
    sim_scores = pd.DataFrame(columns = pattern_weights.columns, index = transaction_weights.columns)
    for t in range(num_tran):
        for p  in range(num_p):
            #features_ind = (~transaction_weights.iloc[:,t].isna()) & (~author_weights.iloc[:,a].isna())
            tran =  transaction_weights.iloc[:, t]
            pattern = pattern_weights.iloc[:, p]
         
            #sim_scores.iloc[t,a] = cos_sim(tran, author)
            sim_scores.iloc[t,p]  = 1 - spatial.distance.cosine(tran, pattern)
    return sim_scores

### Show representative titles for a given author

In [10]:
sim_scores_author = sim_tran_pattern(transaction_weights, author_weights)

In [ ]:
sim_scores_author.to_csv(path+"similarity_scores_of_transaction_to_author.csv", index = False)

In [11]:
author = authors[0]
r = 5
sorted_scores_author = sim_scores_author.sort_values(by = author, ascending = False)
sorted_scores_author.head(10)

,Ralf Steinmetz,Cheng-Wen Wu,Thomas S. Huang,Maja J. Mataric,K. Suzanne Barber,Josef Kittler,Gerald Sommer,Edwin R. Hancock,Masaru Kitsuregawa,Roberto Gorrieri,Nancy E. Reed,Bharat K. Bhargava,Sanjay Kumar Madria,Bill Hancock
1776,0.153945,0.130269,0.137261,0.144066,0.131666,0.136735,0.135691,0.139772,0.144426,0.137263,0.132244,0.130568,0.138665,0.126641
3030,0.133108,0.106321,0.12049,0.117544,0.106823,0.116218,0.111026,0.112547,0.125675,0.111721,0.101806,0.107896,0.113418,0.0986949
1281,0.127576,0.136183,0.139607,0.136626,0.119762,0.121626,0.140759,0.126553,0.121575,0.136748,0.120429,0.144731,0.137064,0.130578
1576,0.126976,0.112431,0.117817,0.124376,0.114144,0.114502,0.123993,0.123203,0.115567,0.125606,0.122432,0.116922,0.125544,0.119862
1127,0.126976,0.112431,0.117817,0.124376,0.114144,0.114502,0.123993,0.123203,0.115567,0.125606,0.122432,0.116922,0.125544,0.119862
631,0.125937,0.134474,0.125845,0.127373,0.129805,0.121333,0.127494,0.136232,0.131797,0.130084,0.138071,0.130276,0.130472,0.135375
70,0.125884,0.101602,0.10723,0.132747,0.098301,0.111255,0.115661,0.103826,0.106593,0.112386,0.0907096,0.107189,0.111364,0.100854
1200,0.125506,0.138523,0.129804,0.134018,0.126893,0.113166,0.125676,0.127234,0.129379,0.126957,0.116624,0.140335,0.127962,0.127896
270,0.12513,0.118359,0.115875,0.121809,0.105217,0.111931,0.122641,0.116085,0.121986,0.110523,0.105918,0.0988239,0.110074,0.113421
3200,0.123899,0.126016,0.123234,0.118538,0.122384,0.110639,0.119142,0.129388,0.118244,0.12278,0.123098,0.12223,0.122955,0.134853


In [12]:
ind = sorted_scores_author.index[0:r]
rep_titles = pd.DataFrame(columns  = [author])
rep_titles[author] = D['title'].iloc[ind]
rep_titles

,Ralf Steinmetz
1776,Domain Name Based Visualization of Web Histori...
3030,Intelligent graphical user interface design ut...
1281,Realistic Force Feedback for Virtual Reality B...
1576,Alignment and Correspondence Using Singular Va...
1127,Symbolic Graph Matching Using the EM Algorithm...


In [ ]:
rep_titles.to_csv(path + 'rep_titles_author_example1.csv', index = False)

### Calculate similarity between the transaction and the title

In [13]:
# Get title list 
titles = context_units.iloc[14:]["pattern"]
titles

14                   xml
15       extend abstract
16              workshop
17                  name
18                weight
             ...        
175              robocup
176             contextu
177            intension
178                 nest
179    distribut resourc
Name: pattern, Length: 166, dtype: object

In [14]:
title_weights = context_weights.iloc[:, 15:]
title_weights

,xml,extend abstract,workshop,name,weight,transmiss,use applic,guest introduct,autonomi,use transform,...,warp,movement,trade,reusabl,phylogeni,robocup,contextu,intension,nest,distribut resourc
0,0.003063,0.003177,0.003095,0.003314,0.003339,0.003415,0.002688,0.003037,0.003091,0.003858,...,0.003227,0.003914,0.003119,0.004322,0.003792,0.003134,0.003914,0.004453,0.003179,0.002649
1,0.003139,0.003244,0.003635,0.002905,0.002963,0.003993,0.003680,0.003095,0.003149,0.003430,...,0.003851,0.002292,0.003688,0.003796,0.003851,0.003743,0.003382,0.003910,0.004390,0.003230
2,0.003063,0.003177,0.003095,0.002847,0.003339,0.003415,0.003135,0.003037,0.003091,0.003363,...,0.003792,0.003331,0.003119,0.003737,0.003792,0.003684,0.003914,0.003851,0.003737,0.002649
3,0.002648,0.003177,0.003095,0.002847,0.002897,0.003415,0.002688,0.002570,0.002617,0.003363,...,0.002690,0.002779,0.003119,0.004322,0.003792,0.003134,0.003331,0.004453,0.003179,0.002649
4,0.003024,0.002801,0.003564,0.002937,0.002915,0.003487,0.003178,0.003149,0.003679,0.003430,...,0.003458,0.002531,0.003241,0.003402,0.004574,0.003880,0.003579,0.004071,0.003940,0.003402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.002528,0.002619,0.002973,0.002311,0.002775,0.002831,0.003013,0.002478,0.002984,0.002785,...,0.003134,0.002190,0.002550,0.003086,0.004262,0.004138,0.003238,0.003743,0.003629,0.003086
176,0.003220,0.003315,0.003233,0.003452,0.003497,0.003554,0.002808,0.003667,0.003212,0.003502,...,0.003914,0.004035,0.003759,0.003858,0.003914,0.003238,0.005328,0.003973,0.003283,0.002738
177,0.003139,0.003244,0.003635,0.003381,0.003415,0.003482,0.003202,0.003095,0.003149,0.003934,...,0.003278,0.003382,0.003178,0.004390,0.004453,0.003743,0.003973,0.005162,0.003796,0.003230
178,0.002993,0.002671,0.003487,0.002792,0.002834,0.003352,0.003532,0.003465,0.003525,0.003300,...,0.004322,0.002222,0.003064,0.003131,0.003737,0.003629,0.003283,0.003796,0.004862,0.003131


In [15]:
start = time.time()
sim_scores_title = sim_tran_pattern(transaction_weights, title_weights)
end = time.time()
print((end-start)/60)

3.8256295760472616


In [ ]:
sim_scores_title.to_csv(path+"similarity_scores_of_transaction_to_title.csv", index = False)

### Show representative titles for a given title

In [16]:
title = titles[10+15] # the same one used for definition
r = 5
sorted_scores_title = sim_scores_title.sort_values(by = title, ascending = False)
sorted_scores_title.head(10)

,xml,extend abstract,workshop,name,weight,transmiss,use applic,guest introduct,autonomi,use transform,...,warp,movement,trade,reusabl,phylogeni,robocup,contextu,intension,nest,distribut resourc
1281,0.124163,0.139666,0.125046,0.116507,0.135496,0.128463,0.131046,0.119077,0.116156,0.120784,...,0.130734,0.124439,0.129522,0.136056,0.131312,0.13815,0.135764,0.133097,0.129804,0.120305
3213,0.123498,0.13078,0.125225,0.115264,0.133305,0.119453,0.12861,0.116979,0.113902,0.117957,...,0.117014,0.115599,0.114864,0.124778,0.128565,0.132928,0.124257,0.130175,0.124763,0.111662
3890,0.123498,0.13078,0.125225,0.115264,0.133305,0.119453,0.12861,0.116979,0.113902,0.117957,...,0.117014,0.115599,0.114864,0.124778,0.128565,0.132928,0.124257,0.130175,0.124763,0.111662
3896,0.123498,0.13078,0.125225,0.115264,0.133305,0.119453,0.12861,0.116979,0.113902,0.117957,...,0.117014,0.115599,0.114864,0.124778,0.128565,0.132928,0.124257,0.130175,0.124763,0.111662
1200,0.121073,0.121008,0.128893,0.1236,0.12237,0.122506,0.135215,0.120199,0.117394,0.126692,...,0.12749,0.116539,0.118569,0.127189,0.128306,0.134279,0.120904,0.136626,0.138477,0.131944
1776,0.130869,0.128926,0.133391,0.146312,0.130471,0.133465,0.12707,0.135219,0.12765,0.145042,...,0.13135,0.157616,0.134113,0.144859,0.130403,0.122214,0.140187,0.147727,0.130125,0.134207
631,0.134363,0.125526,0.134691,0.140264,0.128247,0.132965,0.135313,0.137486,0.12708,0.136177,...,0.143973,0.139256,0.14167,0.127725,0.123429,0.123004,0.142888,0.131018,0.134702,0.140762
3952,0.128391,0.117859,0.122795,0.132571,0.121563,0.125791,0.128825,0.14143,0.144105,0.135961,...,0.147205,0.112176,0.120249,0.12572,0.115102,0.120588,0.112777,0.123048,0.146215,0.130441
3200,0.128391,0.117859,0.122795,0.132571,0.121563,0.125791,0.128825,0.14143,0.144105,0.135961,...,0.147205,0.112176,0.120249,0.12572,0.115102,0.120588,0.112777,0.123048,0.146215,0.130441
2544,0.128391,0.117859,0.122795,0.132571,0.121563,0.125791,0.128825,0.14143,0.144105,0.135961,...,0.147205,0.112176,0.120249,0.12572,0.115102,0.120588,0.112777,0.123048,0.146215,0.130441


In [17]:
ind = sorted_scores_title.index[0:r]
rep_titles = pd.DataFrame(columns  = [title])
rep_titles[title] = D['title'].iloc[ind]
rep_titles

,virtual realiti
1281,Realistic Force Feedback for Virtual Reality B...
3213,Editorial.
3890,Multi-level transaction model for semantic con...
3896,An adaptable constrained locking protocol for ...
1200,Blackboard Segmentation Using Video Image of L...


In [ ]:
rep_titles.to_csv(path + 'rep_titles_title_example1.csv', index = False)